# loading pickled model

In [1]:
import pickle

In [2]:
with open('trained-model.pkl', 'rb') as f:
    model = pickle.load(f)

In [3]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

# live demo

In [4]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
drawing_styles = mp.solutions.drawing_styles

In [ ]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # making prediction
            coords = np.array([[i.x, i.y, i.z] for i in results.multi_hand_landmarks[0].landmark]).flatten()
            X = pd.DataFrame([coords])
            prediction = model.predict(X)[0]
            prediction_proba = model.predict_proba(X)[0]
            
            cv2.rectangle(image,
                          (10, 30 - 30), 
                          (10+len(str(prediction))*20, 30), 
                          (0, 0, 0), -1)
            cv2.putText(image, str(prediction), (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2, cv2.LINE_4)
            
        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


In [7]:
cap.release()